# **Download data from github**

In [1]:
! git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 3651 (delta 13), reused 21 (delta 8), pack-reused 3615
Receiving objects: 100% (3651/3651), 632.29 MiB | 42.82 MiB/s, done.
Resolving deltas: 100% (1461/1461), done.
Checking out files: 100% (1164/1164), done.


# **create a folder named dataset and ...**

In [2]:
! mkdir dataset
! mkdir /content/dataset/train
! mkdir /content/dataset/train/covid
! mkdir /content/dataset/train/normal
! mkdir /content/dataset/test
! mkdir /content/dataset/test/covid
! mkdir /content/dataset/test/normal

# **Download data from kaggle**
from kaggle - my account press "create new token api". upload downloaded file here.

In [3]:
! pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"golabiabi","key":"ccaab0263258794f607a2b32e397be80"}'}

In [4]:
# ******* make root/kaggle folder **********
! mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# ************** download data **********
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
# !unzip
!unzip -q /content/chest-xray-pneumonia.zip

 99% 2.27G/2.29G [00:30<00:00, 41.8MB/s]
100% 2.29G/2.29G [00:30<00:00, 79.8MB/s]


# **extract all covid cases**

In [5]:
import pandas as pd

csvPath = '/content/covid-chestxray-dataset/metadata.csv'
df = pd.read_csv(csvPath)

In [6]:
df.__len__()

941

In [7]:
p_id = 0
cov_fn = []
allfiles = []

for (i, row) in df.iterrows():
	n_id = row["patientid"]

	if n_id != p_id and len(cov_fn)>0 and p_id != 0:
		allfiles.append(cov_fn)
		cov_fn = []
	if row["finding"] == "Pneumonia/Viral/COVID-19" and row["view"] == "PA":
		cov_fn.append(row["filename"])
		p_id = row["patientid"]

# **split covid data to train and test**

In [8]:
from sklearn.model_selection import train_test_split
	
x_train_c, x_test_c = train_test_split(allfiles, test_size=0.20, random_state=23)

# **save data to folders**

In [9]:
import shutil

for img in sum(x_train_c, []):
    src = '/content/covid-chestxray-dataset/images/' + img
    dst = '/content/dataset/train/covid/' + img
    shutil.copy2(src, dst)

for img in sum(x_test_c, []):
    src = '/content/covid-chestxray-dataset/images/' + img
    dst = '/content/dataset/test/covid/' + img
    shutil.copy2(src, dst)

In [10]:
import os
import cv2
import random

n_samples = len(sum(allfiles, []))
kaggle_data_path = '/content/chest_xray/train/NORMAL/'
output_path_train = '/content/dataset/train/normal/'
output_path_test = '/content/dataset/test/normal/'

filenames = os.listdir(kaggle_data_path)
random.seed(42)
filenames = random.sample(filenames, len(filenames))
for i in range(n_samples):
    n_image = cv2.imread(kaggle_data_path + filenames[i])
    if i < sum(x_train_c, []).__len__():
        cv2.imwrite(output_path_train + filenames[i], n_image)
    else:
        cv2.imwrite(output_path_test + filenames[i], n_image)

# **prepare data**

In [11]:
import os 
import numpy as np 

path = "/content/dataset" 
x_train_n = [] 
x_train_c = [] 
x_test_n = [] 
x_test_c = [] 

for p in os.listdir(path+ '/train/normal/'): 
    x_train_n.append(cv2.imread(path + '/train/normal/' + p)) 
for p in os.listdir(path+ '/train/covid/'): 
    x_train_c.append(cv2.imread(path + '/train/covid/' + p)) 
for p in os.listdir(path+ '/test/normal/'): 
    x_test_n.append(cv2.imread(path + '/test/normal/' + p)) 
for p in os.listdir(path+ '/test/covid/'): 
    x_test_c.append(cv2.imread(path + '/test/covid/' + p))

In [12]:
for i in range(len(x_train_n)):
    x_train_n[i] = cv2.resize(x_train_n[i], (224, 224))
    x_train_c[i] = cv2.resize(x_train_c[i], (224, 224))

for i in range(len(x_test_n)):
    x_test_n[i] = cv2.resize(x_test_n[i], (224, 224))
    x_test_c[i] = cv2.resize(x_test_c[i], (224, 224))

In [13]:
x_train_n = np.array(x_train_n)
x_train_c = np.array(x_train_c)
x_test_n = np.array(x_test_n)
x_test_c = np.array(x_test_c)

In [14]:
print(x_train_n.shape) 
print(x_train_c.shape) 
print(x_test_n.shape) 
print(x_test_c.shape)

(148, 224, 224, 3)
(148, 224, 224, 3)
(47, 224, 224, 3)
(47, 224, 224, 3)


In [15]:
x_train = np.concatenate((x_train_n, x_train_c))/255.0
x_test = np.concatenate((x_test_n, x_test_c))/255.0

In [16]:
y_train_n = np.zeros(x_train_n.shape[0])
y_train_c = np.ones(x_train_c.shape[0])
y_test_n = np.zeros(x_test_n.shape[0])
y_test_c = np.ones(x_test_c.shape[0])

In [17]:
y_train = np.concatenate((y_train_n, y_train_c))
y_test = np.concatenate((y_test_n, y_test_c))

In [18]:
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

In [19]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(296, 224, 224, 3)
(94, 224, 224, 3)
(296, 1)
(94, 1)


# **define model**

In [20]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [21]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-3
EPOCHS = 50
BS = 8

In [22]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")

In [23]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

58892288/58889256 [==============================] - 0s 0us/step


In [24]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(x_train, y_train, batch_size=BS),
	steps_per_epoch=len(x_train) // BS,
	validation_data=(x_test, y_test),
	validation_steps=len(x_test) // BS,
	epochs=EPOCHS)

[INFO] compiling model...
[INFO] training head...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
37/37 [==============================] - 5s 139ms/step - loss: 0.6236 - accuracy: 0.6520 - val_loss: 0.4694 - val_accuracy: 0.9468
Epoch 2/50
37/37 [==============================] - 4s 108ms/step - loss: 0.4234 - accuracy: 0.8480 - val_loss: 0.3183 - val_accuracy: 0.9468
Epoch 3/50
37/37 [==============================] - 4s 108ms/step - loss: 0.3176 - accuracy: 0.9155 - val_loss: 0.2423 - val_accuracy: 0.9574
Epoch 4/50
37/37 [==============================] - 4s 108ms/step - loss: 0.2675 - accuracy: 0.9122 - val_loss: 0.2038 - val_accuracy: 0.9468
Epoch 5/50
37/37 [==============================] - 4s 108ms/step - loss: 0.2471 - accuracy: 0.9189 - val_loss: 0.1899 - val_accuracy: 0.9574
Epoch 6/50
37/37 [==============================] - 4s 108ms/step - loss: 0.2090 - accuracy: 0.9392 - val_loss: 0.1580 - val_accuracy: 0.9574
Epoch 7/50
37/37 [====

# **prediction**

In [25]:
model.predict(x_test)

array([[3.3645108e-04],
       [1.2787229e-04],
       [3.0687204e-04],
       [1.8594293e-04],
       [8.8789128e-04],
       [2.2062780e-03],
       [8.5091895e-05],
       [2.3272367e-04],
       [6.8524624e-03],
       [2.3365160e-03],
       [4.6214741e-04],
       [4.8456270e-01],
       [2.1608867e-04],
       [9.8477978e-01],
       [3.7198868e-05],
       [5.3255004e-04],
       [3.3552865e-03],
       [3.3389160e-04],
       [9.6598919e-04],
       [9.8803849e-04],
       [6.0224649e-04],
       [6.5070181e-04],
       [3.8730315e-04],
       [2.2554809e-01],
       [3.8252815e-04],
       [4.0165377e-03],
       [1.2528970e-03],
       [9.4746792e-05],
       [4.5105047e-04],
       [2.0995762e-03],
       [4.0024426e-04],
       [2.2839874e-03],
       [7.5507886e-04],
       [4.5164018e-03],
       [1.0044893e-02],
       [4.7172961e-04],
       [1.3611122e-04],
       [7.9747280e-03],
       [4.3664771e-04],
       [7.0768646e-03],
       [3.5995257e-04],
       [2.333210

In [26]:
# predicts = model.predict(x_test)
th = np.linspace(0.2, 0.8, 7)
all_cms = []
all_sens = []
all_spec = []
all_acc = []
for t in th:
    preds = model.predict(x_test)
    preds[preds > t] = 1
    preds[preds < t] = 0
    cm = confusion_matrix(y_test, preds)
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    all_acc.append(acc)
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    all_cms.append(cm)
    all_sens.append(sensitivity)
    all_spec.append(specificity)
print('best accuracy: ', all_acc[np.argmax(all_acc)])
print('threshold: ', th[np.argmax(all_acc)])
print('confusion matrix: ', '\n', all_cms[np.argmax(all_acc)])
print('sensitivity: ', all_sens[np.argmax(all_acc)])
print('specificity: ', all_spec[np.argmax(all_acc)])



best accuracy:  0.9893617021276596
threshold:  0.5
confusion matrix:  
 [[46  1]
 [ 0 47]]
sensitivity:  0.9787234042553191
specificity:  1.0
